<a href="https://colab.research.google.com/github/tanay2001/Amazon_rec_engine/blob/master/item_cf_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np

In [2]:
import pandas as pd
import json
import gzip
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
import re
import string
import nltk


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles/meta_Video_Games.json.gz

--2020-05-14 09:05:40--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles/meta_Video_Games.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80222877 (77M) [application/octet-stream]
Saving to: ‘meta_Video_Games.json.gz’

meta_Video_Games.js 100%[===================>]  76.51M  5.24MB/s    in 15s     

2020-05-14 09:05:55 (5.14 MB/s) - ‘meta_Video_Games.json.gz’ saved [80222877/80222877]



In [0]:
data = []
with gzip.open('meta_Video_Games.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))

In [0]:
df = pd.DataFrame.from_dict(data)


In [27]:
df.head()

,category,title,image,brand,rank,main_cat,asin,description,also_buy,also_view,price,feature,date,tech1,details,similar_item,tech2
0,"[Video Games, PC, Games]",Reversi Sensory Challenger,[https://images-na.ssl-images-amazon.com/image...,Fidelity Electronics,">#2,623,937 in Toys & Games (See Top 100 in To...",Toys & Games,0042000742,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[Video Games, Xbox 360, Games, ]",Medal of Honor: Warfighter - Includes Battlefi...,[https://images-na.ssl-images-amazon.com/image...,by\n \n EA Games,">#67,231 in Video Games (See Top 100 in Video ...",Video Games,0078764343,[Brand new sealed!],[B00PADROYW],"[B0050SY5BM, B072NQJCW5, B000TI836G, B002SRSQ7...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"[Video Games, Retro Gaming & Microconsoles, Su...",street fighter 2 II turbo super nintendo snes ...,NaN,Nintendo,">#134,433 in Video Games (See Top 100 in Video...",Video Games,0276425316,NaN,NaN,NaN,$0.72,NaN,NaN,NaN,NaN,NaN,NaN
3,"[Video Games, Xbox 360, Accessories, Controlle...",Xbox 360 MAS STICK,[https://images-na.ssl-images-amazon.com/image...,by\n \n MAS SYSTEMS,">#105,263 in Video Games (See Top 100 in Video...",Video Games,0324411812,[MAS's Pro Xbox 360 Stick (Perfect 360 Stick) ...,NaN,NaN,NaN,[Original PCB used from Xbox 360 Control Pad (...,NaN,NaN,NaN,NaN,NaN
4,"[Video Games, PC, Games, , Grades 2-12, Spelli...",Phonics Alive! 3: The Speller,[https://images-na.ssl-images-amazon.com/image...,by\n \n Advanced Software Pty. Ltd.,">#92,397 in Video Games (See Top 100 in Video ...",Video Games,0439335310,"[Phonics Alive! 3, The Speller teaches student...",NaN,[B000BCZ7U0],NaN,"[Grades 2-12, Spelling Program, Teaches Spelli...",NaN,NaN,NaN,NaN,NaN


In [0]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Video_Games_5.json.gz

--2020-05-14 09:02:42--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Video_Games_5.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 154050105 (147M) [application/octet-stream]
Saving to: ‘Video_Games_5.json.gz’

Video_Games_5.json. 100%[===================>] 146.91M  5.09MB/s    in 29s     

2020-05-14 09:03:11 (5.00 MB/s) - ‘Video_Games_5.json.gz’ saved [154050105/154050105]



In [0]:
data2 = []
with gzip.open('Video_Games_5.json.gz') as f:
    for l in f:
        data2.append(json.loads(l.strip()))

In [0]:
df2 = pd.DataFrame.from_dict(data2)
df2 = df2[["reviewerID","asin","reviewText", "summary" ,"overall"]]
df2.drop_duplicates(keep=False , inplace = True)

In [30]:
df2.shape

(450756, 5)

In [0]:
df3 = pd.merge(df, df2, on="asin", how='right')

In [0]:
df4 = df3.groupby("asin").agg({"overall":"mean", "reviewerID":"count"})
df4["avg_rating"]= df4.overall
df4["Count"]= df4.reviewerID

In [0]:
df5 = pd.merge(df3, df4, on="asin", how='inner')

In [0]:

df5 = df5[["category","title","asin","reviewerID_x","overall_x","reviewText","summary","avg_rating","Count","brand","feature","description"]]

In [35]:
df5.head()

,category,title,asin,reviewerID_x,overall_x,reviewText,summary,avg_rating,Count,brand,feature,description
0,"[Video Games, PC, Games]",Anno 2070,0700026657,A1HP7NVNPFMA4N,5.0,"This game is a bit hard to get the hang of, bu...",but when you do it's great.,3.692308,13,Ubisoft,[A new era: while adhering to the fundamentals...,[ANNO 2070BRAND NEW - IN STOCKDVD Rom Software...
1,"[Video Games, PC, Games]",Anno 2070,0700026657,A1JGAP0185YJI6,4.0,I played it a while but it was alright. The st...,"But in spite of that it was fun, I liked it",3.692308,13,Ubisoft,[A new era: while adhering to the fundamentals...,[ANNO 2070BRAND NEW - IN STOCKDVD Rom Software...
2,"[Video Games, PC, Games]",Anno 2070,0700026657,A1YJWEXHQBWK2B,3.0,ok game.,Three Stars,3.692308,13,Ubisoft,[A new era: while adhering to the fundamentals...,[ANNO 2070BRAND NEW - IN STOCKDVD Rom Software...
3,"[Video Games, PC, Games]",Anno 2070,0700026657,A2204E1TH211HT,2.0,"found the game a bit too complicated, not what...",Two Stars,3.692308,13,Ubisoft,[A new era: while adhering to the fundamentals...,[ANNO 2070BRAND NEW - IN STOCKDVD Rom Software...
4,"[Video Games, PC, Games]",Anno 2070,0700026657,A2RF5B5H74JLPE,5.0,"great game, I love it and have played it since...",love this game,3.692308,13,Ubisoft,[A new era: while adhering to the fundamentals...,[ANNO 2070BRAND NEW - IN STOCKDVD Rom Software...


In [0]:
df7 = df5.groupby("reviewerID_x").agg({"asin":"count"})
df7["user_count"] = df7.asin

In [0]:
df6 = pd.merge(df5, df7, on="reviewerID_x", how='inner')

In [93]:
df6.head()

,category,title,asin_x,reviewerID_x,overall_x,reviewText,summary,avg_rating,Count,brand,feature,description,asin_y,user_count
0,"[Video Games, PC, Games]",Anno 2070,0700026657,A1HP7NVNPFMA4N,5.0,"This game is a bit hard to get the hang of, bu...",but when you do it's great.,3.692308,13,Ubisoft,[A new era: while adhering to the fundamentals...,[ANNO 2070BRAND NEW - IN STOCKDVD Rom Software...,27,27
1,"[Video Games, Retro Gaming & Microconsoles, Ni...",Hexen,B00000JMWB,A1HP7NVNPFMA4N,5.0,This is great for the old school game lovers.,Five Stars,4.111111,18,by\n \n Atari,[hexen],[Like New Condition! Cart Only. FAST & FREE SH...,27,27
2,"[Video Games, Retro Gaming & Microconsoles, Ni...",Hexen,B00000JMWB,A1HP7NVNPFMA4N,5.0,This is great for the old school game lovers.,Five Stars,4.111111,18,by\n \n Atari,[hexen],[Like New Condition! Cart Only. FAST & FREE SH...,27,27
3,"[Video Games, PC, Accessories, Controllers, Ga...",Logitech Dual Action Game Pad USB,B0000ALFCI,A1HP7NVNPFMA4N,2.0,"This controller stemmed like a good idea, but ...",This controller stemmed like a good idea,3.525000,80,Logitech,[Start playing right away with plug-and-play c...,"[Dual Gamepad,w/ Programmable Joysticks,Black/...",27,27
4,"[Video Games, PC, Accessories, Controllers, Ga...",Logitech Dual Action Game Pad USB,B0000ALFCI,A1HP7NVNPFMA4N,2.0,"This controller stemmed like a good idea, but ...",This controller stemmed like a good idea,3.525000,80,Logitech,[Start playing right away with plug-and-play c...,"[Dual Gamepad,w/ Programmable Joysticks,Black/...",27,27


In [0]:
df2=0
df3=0
df4=0
df5=0

In [98]:
df_final = df6[df6.Count > 25][df6.user_count>15]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [99]:
df_final2  =df_final.drop(["reviewText","summary","feature","description","asin_y","category", "brand","user_count","Count","avg_rating"], axis = 1)
df_final2.head()

,title,asin_x,reviewerID_x,overall_x
3,Logitech Dual Action Game Pad USB,B0000ALFCI,A1HP7NVNPFMA4N,2.0
4,Logitech Dual Action Game Pad USB,B0000ALFCI,A1HP7NVNPFMA4N,2.0
5,Wii,B0009VXBAQ,A1HP7NVNPFMA4N,5.0
6,Mario & Luigi: Partners In Time,B000B8J7K0,A1HP7NVNPFMA4N,3.0
7,Super Mario Galaxy,B000FQ9QVI,A1HP7NVNPFMA4N,5.0


In [100]:
df_final2.shape

(131577, 4)

In [0]:
df_user_item = df_final2.drop_duplicates( )

In [102]:
df_user_item.shape

(99714, 4)

In [0]:
df_user_item2 = df_user_item.drop_duplicates(subset=["asin_x","reviewerID_x"])

In [104]:
df_user_item2.head()

,title,asin_x,reviewerID_x,overall_x
3,Logitech Dual Action Game Pad USB,B0000ALFCI,A1HP7NVNPFMA4N,2.0
5,Wii,B0009VXBAQ,A1HP7NVNPFMA4N,5.0
6,Mario & Luigi: Partners In Time,B000B8J7K0,A1HP7NVNPFMA4N,3.0
7,Super Mario Galaxy,B000FQ9QVI,A1HP7NVNPFMA4N,5.0
8,Age Of Mythology: Titans - PC,B000P5BSUQ,A1HP7NVNPFMA4N,5.0


In [0]:
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [0]:
mat = df_user_item2.pivot(index='asin_x', columns='reviewerID_x', values='overall_x').fillna(0)

In [0]:
mat_sparce = mat.fillna(0)

In [107]:
mat_sparce

reviewerID_x,A0266076X6KPZ6CCHGVS,A0380485C177Q6QQNJIX,A0685888WB02Q69S553P,A100JCBNALJFAW,A1027EV8A9PV1O,A102GA81P1MWWZ,A103B6MQ5IF2BK,A103KKI1Y4TFNQ,A103KP5IOW1SFI,A10566TEP2SO6I,A105GWGM7PDAI2,A105S56ODHGJEK,A10795M6XA74JN,A1084SNUEXOQJY,A1085T5S1400VA,A109IF09XK3YNQ,A109LWN9DUGPDP,A10ABCICVBQW92,A10C84Y38RT22P,A10DIDVYDN2Q5Z,A10DZNGD2T2SP3,A10EAONPHFORLH,A10EM4DKTUAR7W,A10EN9IKKOEY97,A10H5ID8FBMTIG,A10HHM2684NZD2,A10IJF7UD9I86G,A10JO6RV8UFYBV,A10JPTQTDNTV43,A10L7QIDE9UF11,A10MHIN0CZONJB,A10T653BS1I4IN,A10TFL9E75GUE2,A10UUYDN5NY2GB,A10VEFED62UPM9,A10VQUCFXVTXHG,A10VW7VC4X2OI6,A10W2QTBTVV6RQ,A10WUIJQEOXDJ7,A10Y058K7B96C6,...,AZ6PUO2C4HDWH,AZ7PDE0NWFS2B,AZ7X1ED2UGEL2,AZ85AMRA3BIOJ,AZ8K0NSCMPKSX,AZ8UG2KJ7UWKV,AZ9JWGE1UGKZA,AZA595ZPIG240,AZAC7B4HD5ZVC,AZAH84SERW5GR,AZAVOS2WXDF23,AZB0MVRWKL8TE,AZDIS9H8DY7RY,AZEMV7Z9YDR79,AZH2GUV4NXHOW,AZH7PD34I849I,AZIIM0UOCTSHV,AZJ88OY287AV2,AZJEY6KBCBFG0,AZJL03R8NDQ7C,AZKB5T3T2LTE8,AZKW2LP60H9WA,AZL8QJZ9XPQH0,AZMP1U4ZYC3F9,AZNCDOLHNG9GO,AZNKIND8TQ588,AZOD1JRXNWL7G,AZP1HKS4XINS0,AZPMU427WH0RB,AZQ7O73EAS3VI,AZQ8HGO3Z4T,AZQYAXMAC6AJ4,AZUW77T88WC1S,AZVCNOOQK36DH,AZXHK8IO25FL6,AZXWP83KBEA7N,AZYG5R26NUP70,AZYU8M791SIFC,AZZ1KF8RAO1BR,AZZ3GFL1HTBLY
asin_x,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0700099867,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0804161380,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6050036071,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9625990674,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9629971372,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B01H2DKIDG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B01H3NU4OS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B01H482N6E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [0]:

id_asin={}
c=0
for i in mat_sparce.index:
  id_asin[i]=c
  c=c+1

In [0]:
def modelling(df_mat):
  model_knn = NearestNeighbors(metric='cosine', algorithm='auto', n_neighbors=20, n_jobs=-1)
  model_knn.fit(df_mat)
  distances, indices = model_knn.kneighbors(df_mat)
  simi = []
  for row in distances:
    d=[]
    for i in row:
      d.append(1-i)
    simi.append(d)
  return simi, indices

In [0]:


def get_recommendations(user,df_data):
  history = df_data[user][df_data[user] >=4]
  prd_history = history.index
  k=10
  t={}
  simi2, indices2 = modelling(df_data) 
  for prd in prd_history:
      a=0
      i = id_asin[prd]
      #print(i)
      similar_prds_indices = indices2[i]
      similarity_score = simi2[i]
      for item in similar_prds_indices[:k]:
          #print(item)
          rating = history.loc[prd]/5
          #print(similarity_score[item-1])
          if item not in t.keys():
            t[item]= rating*similarity_score[a]
          else:
            t[item]= t[item] +rating*similarity_score[a]
          a=a+1
      
  recommendation = sorted(t.items(), key = lambda x:(x[1]), reverse = True)
  return recommendation, prd_history

In [0]:

def creation(rec,prd_hist):
  k=10
  prd_history_id = [id_asin[prd] for prd in prd_hist ]
  final_rec =[]
  for key in rec:
      if key[0]  not in prd_history_id:
        #print(list(id_asin.keys())[list(id_asin.values()).index(key[0])])
        final_rec.append(key)

  final_rec2 = [list(id_asin.keys())[list(id_asin.values()).index(val[0])] for val in final_rec[:k]]
  return final_rec2


In [0]:
user_list = mat_sparce.columns
k=10

In [147]:

hit_rate =0
for user in user_list[0:10]:
      print(user)
  
      test_data =  mat_sparce[user][mat_sparce[user]>=4].index[0]
      mat_sparce[user][mat_sparce[user]>=4][0]=0

      rec_li, phist = get_recommendations(user, mat_sparce)

      recommendation_list = creation(rec_li,phist)
      #print(recommendation_list)
      if test_data in recommendation_list:
        hit_rate = hit_rate +1
      print(hit_rate)
print(hit_rate/100)

A0266076X6KPZ6CCHGVS
0
A0380485C177Q6QQNJIX
0
A0685888WB02Q69S553P
0
A100JCBNALJFAW
0
A1027EV8A9PV1O
0
A102GA81P1MWWZ
0
A103B6MQ5IF2BK
0
A103KKI1Y4TFNQ
0
A103KP5IOW1SFI
0
A10566TEP2SO6I
0
0.0


In [146]:
hit_rate

0